In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction import FeatureHasher

In [2]:
df_wines = pd.read_csv("XWines_Slim_1K_wines.csv")
df_ratings = pd.read_csv("XWines_Slim_150K_ratings.csv")

C:\Users\vince\AppData\Local\Temp\ipykernel_12440\174622797.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_ratings = pd.read_csv("XWines_Slim_150K_ratings.csv")


In [3]:
df_wines.head()

,WineID,WineName,Type,Elaborate,Grapes,Harmonize,ABV,Body,Acidity,Code,Country,RegionID,RegionName,WineryID,WineryName,Website,Vintages
0,100001,Espumante Moscatel,Sparkling,Varietal/100%,['Muscat/Moscato'],"['Pork', 'Rich Fish', 'Shellfish']",7.5,Medium-bodied,High,BR,Brazil,1001,Serra Gaúcha,10001,Casa Perini,http://www.vinicolaperini.com.br,"[2020, 2019, 2018, 2017, 2016, 2015, 2014, 201..."
1,100002,Ancellotta,Red,Varietal/100%,['Ancellotta'],"['Beef', 'Barbecue', 'Codfish', 'Pasta', 'Pizz...",12.0,Medium-bodied,Medium,BR,Brazil,1001,Serra Gaúcha,10001,Casa Perini,http://www.vinicolaperini.com.br,"[2016, 2015, 2014, 2013, 2012, 2011, 2010, 200..."
2,100003,Cabernet Sauvignon,Red,Varietal/100%,['Cabernet Sauvignon'],"['Beef', 'Lamb', 'Poultry']",12.0,Full-bodied,High,BR,Brazil,1001,Serra Gaúcha,10002,Castellamare,https://www.emporiocastellamare.com.br,"[2021, 2020, 2019, 2018, 2017, 2016, 2015, 201..."
3,100005,Maison de Ville Cabernet-Merlot,Red,Assemblage/Bordeaux Red Blend,"['Cabernet Sauvignon', 'Merlot']","['Beef', 'Lamb', 'Game Meat', 'Poultry']",11.0,Full-bodied,Medium,BR,Brazil,1001,Serra Gaúcha,10000,Aurora,http://www.vinicolaaurora.com.br,"[2021, 2020, 2019, 2018, 2017, 2016, 2015, 201..."
4,100007,Do Lugar Moscatel Espumantes,Sparkling,Varietal/100%,['Muscat/Moscato Bianco'],"['Pork', 'Rich Fish', 'Shellfish']",7.5,Medium-bodied,High,BR,Brazil,1001,Serra Gaúcha,10012,Dal Pizzol,http://www.dalpizzol.com.br,"[2018, 2017, 2016, 2015, 2014, 2013, 2012, 201..."


In [4]:
df_ratings.head()

,RatingID,UserID,WineID,Vintage,Rating,Date
0,143,1356810,103471,1950,4.5,2021-11-02 20:52:59
1,199,1173759,111415,1951,5.0,2015-08-20 17:46:26
2,348,1164877,111395,1952,5.0,2020-11-13 05:40:26
3,374,1207665,111433,1953,5.0,2017-05-05 06:44:13
4,834,1075841,111431,1955,5.0,2016-09-14 20:18:38


In [5]:
wine_ids = df_wines['WineID'].copy()

In [6]:
# --- Create DataFrame for Preference Matching (Before Heavy Transformation) ---
print("Creating DataFrame for preference matching...")
# Define columns needed for direct preference matching
preference_cols_list = ['WineID', 'WineName', 'Type', 'Grapes', 'Body', 'Acidity', 'Country', 'RegionName', 'WineryName', 'Elaborate']
valid_preference_cols = [col for col in preference_cols_list if col in df_wines.columns]
if valid_preference_cols:
    df_for_prefs = df_wines[valid_preference_cols].copy()
    print(f"Preference DataFrame created with columns: {valid_preference_cols}")
else:
    df_for_prefs = pd.DataFrame() # Empty DataFrame if no relevant columns found
    print("Warning: No suitable columns found for preference matching DataFrame.")


Creating DataFrame for preference matching...
Preference DataFrame created with columns: ['WineID', 'WineName', 'Type', 'Grapes', 'Body', 'Acidity', 'Country', 'RegionName', 'WineryName', 'Elaborate']


In [7]:
preprocessing = ['WineName', 'Type', 'Elaborate', 'Body', 'Acidity', 'Country', 'RegionName', 'WineryName']
for col in preprocessing:
    df_wines[col] = df_wines[col].str.lower()

In [8]:
df_wines.head()

,WineID,WineName,Type,Elaborate,Grapes,Harmonize,ABV,Body,Acidity,Code,Country,RegionID,RegionName,WineryID,WineryName,Website,Vintages
0,100001,espumante moscatel,sparkling,varietal/100%,['Muscat/Moscato'],"['Pork', 'Rich Fish', 'Shellfish']",7.5,medium-bodied,high,BR,brazil,1001,serra gaúcha,10001,casa perini,http://www.vinicolaperini.com.br,"[2020, 2019, 2018, 2017, 2016, 2015, 2014, 201..."
1,100002,ancellotta,red,varietal/100%,['Ancellotta'],"['Beef', 'Barbecue', 'Codfish', 'Pasta', 'Pizz...",12.0,medium-bodied,medium,BR,brazil,1001,serra gaúcha,10001,casa perini,http://www.vinicolaperini.com.br,"[2016, 2015, 2014, 2013, 2012, 2011, 2010, 200..."
2,100003,cabernet sauvignon,red,varietal/100%,['Cabernet Sauvignon'],"['Beef', 'Lamb', 'Poultry']",12.0,full-bodied,high,BR,brazil,1001,serra gaúcha,10002,castellamare,https://www.emporiocastellamare.com.br,"[2021, 2020, 2019, 2018, 2017, 2016, 2015, 201..."
3,100005,maison de ville cabernet-merlot,red,assemblage/bordeaux red blend,"['Cabernet Sauvignon', 'Merlot']","['Beef', 'Lamb', 'Game Meat', 'Poultry']",11.0,full-bodied,medium,BR,brazil,1001,serra gaúcha,10000,aurora,http://www.vinicolaaurora.com.br,"[2021, 2020, 2019, 2018, 2017, 2016, 2015, 201..."
4,100007,do lugar moscatel espumantes,sparkling,varietal/100%,['Muscat/Moscato Bianco'],"['Pork', 'Rich Fish', 'Shellfish']",7.5,medium-bodied,high,BR,brazil,1001,serra gaúcha,10012,dal pizzol,http://www.dalpizzol.com.br,"[2018, 2017, 2016, 2015, 2014, 2013, 2012, 201..."


In [9]:
preprocessing2 = ['Grapes', 'Harmonize']

for col in preprocessing2:
    if col in df_wines.columns:
        # 1. Fill NaN with empty string FIRST, then ensure string type
        #    This prevents np.nan from becoming the string 'nan'
        series = df_wines[col].fillna('').astype(str) # <-- Swapped order here

        # 2. Remove unwanted characters
        series = series.str.replace('[', '', regex=False)
        series = series.str.replace(']', '', regex=False)
        series = series.str.replace("'", '', regex=False)
        series = series.str.replace('/', '', regex=False) # Remove slash

        # 3. Split into a Series of lists
        series_list = series.str.split(',')

        # 4. Apply cleaning (strip) AND lowercasing to items WITHIN each list
        cleaned_series_list = series_list.apply(
            lambda lst: [item.strip().lower() for item in lst if isinstance(item, str)] # Apply strip() and lower()
            if isinstance(lst, list) else # Check if the element is actually a list
            lst # Handle potential non-list elements
        )

        # 5. Assign back, converting potential [''] resulting from empty strings OR original NaNs to []
        df_wines[col] = cleaned_series_list.apply(lambda x: [] if x == [''] else x)

    else:
         print(f"Warning: Column '{col}' not found in DataFrame.")

In [10]:
df_wines.head()

,WineID,WineName,Type,Elaborate,Grapes,Harmonize,ABV,Body,Acidity,Code,Country,RegionID,RegionName,WineryID,WineryName,Website,Vintages
0,100001,espumante moscatel,sparkling,varietal/100%,[muscatmoscato],"[pork, rich fish, shellfish]",7.5,medium-bodied,high,BR,brazil,1001,serra gaúcha,10001,casa perini,http://www.vinicolaperini.com.br,"[2020, 2019, 2018, 2017, 2016, 2015, 2014, 201..."
1,100002,ancellotta,red,varietal/100%,[ancellotta],"[beef, barbecue, codfish, pasta, pizza, cheese]",12.0,medium-bodied,medium,BR,brazil,1001,serra gaúcha,10001,casa perini,http://www.vinicolaperini.com.br,"[2016, 2015, 2014, 2013, 2012, 2011, 2010, 200..."
2,100003,cabernet sauvignon,red,varietal/100%,[cabernet sauvignon],"[beef, lamb, poultry]",12.0,full-bodied,high,BR,brazil,1001,serra gaúcha,10002,castellamare,https://www.emporiocastellamare.com.br,"[2021, 2020, 2019, 2018, 2017, 2016, 2015, 201..."
3,100005,maison de ville cabernet-merlot,red,assemblage/bordeaux red blend,"[cabernet sauvignon, merlot]","[beef, lamb, game meat, poultry]",11.0,full-bodied,medium,BR,brazil,1001,serra gaúcha,10000,aurora,http://www.vinicolaaurora.com.br,"[2021, 2020, 2019, 2018, 2017, 2016, 2015, 201..."
4,100007,do lugar moscatel espumantes,sparkling,varietal/100%,[muscatmoscato bianco],"[pork, rich fish, shellfish]",7.5,medium-bodied,high,BR,brazil,1001,serra gaúcha,10012,dal pizzol,http://www.dalpizzol.com.br,"[2018, 2017, 2016, 2015, 2014, 2013, 2012, 201..."


In [11]:
df_wines['ABV'] = df_wines['ABV'].astype(float)

In [12]:
df_wines.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1007 entries, 0 to 1006
Data columns (total 17 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   WineID      1007 non-null   int64  
 1   WineName    1007 non-null   object 
 2   Type        1007 non-null   object 
 3   Elaborate   1007 non-null   object 
 4   Grapes      1007 non-null   object 
 5   Harmonize   1007 non-null   object 
 6   ABV         1007 non-null   float64
 7   Body        1007 non-null   object 
 8   Acidity     1007 non-null   object 
 9   Code        1007 non-null   object 
 10  Country     1007 non-null   object 
 11  RegionID    1007 non-null   int64  
 12  RegionName  1007 non-null   object 
 13  WineryID    1007 non-null   int64  
 14  WineryName  1007 non-null   object 
 15  Website     900 non-null    object 
 16  Vintages    1007 non-null   object 
dtypes: float64(1), int64(3), object(13)
memory usage: 133.9+ KB


In [13]:
categorical_features = ['Type', 'Elaborate', 'Body', 'Acidity', 'Country']
df_wines = pd.get_dummies(df_wines, columns=categorical_features)

In [14]:
df_wines.head()

,WineID,WineName,Grapes,Harmonize,ABV,Code,RegionID,RegionName,WineryID,WineryName,...,Country_russia,Country_slovenia,Country_south africa,Country_spain,Country_switzerland,Country_turkey,Country_ukraine,Country_united kingdom,Country_united states,Country_uruguay
0,100001,espumante moscatel,[muscatmoscato],"[pork, rich fish, shellfish]",7.5,BR,1001,serra gaúcha,10001,casa perini,...,False,False,False,False,False,False,False,False,False,False
1,100002,ancellotta,[ancellotta],"[beef, barbecue, codfish, pasta, pizza, cheese]",12.0,BR,1001,serra gaúcha,10001,casa perini,...,False,False,False,False,False,False,False,False,False,False
2,100003,cabernet sauvignon,[cabernet sauvignon],"[beef, lamb, poultry]",12.0,BR,1001,serra gaúcha,10002,castellamare,...,False,False,False,False,False,False,False,False,False,False
3,100005,maison de ville cabernet-merlot,"[cabernet sauvignon, merlot]","[beef, lamb, game meat, poultry]",11.0,BR,1001,serra gaúcha,10000,aurora,...,False,False,False,False,False,False,False,False,False,False
4,100007,do lugar moscatel espumantes,[muscatmoscato bianco],"[pork, rich fish, shellfish]",7.5,BR,1001,serra gaúcha,10012,dal pizzol,...,False,False,False,False,False,False,False,False,False,False


In [15]:
grapes_text = df_wines['Grapes'].apply(lambda lst: ' '.join(lst))
harmonize_text = df_wines['Harmonize'].apply(lambda lst: ' '.join(lst))
corpus = grapes_text + ' ' + harmonize_text

vect = TfidfVectorizer()
tfidf_matrix = vect.fit_transform(corpus)


In [16]:
scaler = StandardScaler()
df_wines['ABV'] = scaler.fit_transform(df_wines[['ABV']])

In [17]:
df_wines['RegionName'].value_counts()

RegionName
serra gaúcha            32
mendoza                 32
central valley (cl)     29
napa valley             27
douro                   22
                        ..
carignano del sulcis     1
bardolino classico       1
fiano di avellino        1
collio                   1
südsteiermark            1
Name: count, Length: 324, dtype: int64

In [18]:
N_FEATURES_HASH = 16
hasher = FeatureHasher(n_features=N_FEATURES_HASH, input_type='string', alternate_sign=False)
region_feature_values = [[region] for region in df_wines['RegionName']]
hashed_features_sparse = hasher.fit_transform(region_feature_values)
hashed_feature_columns = [f'RegionHash_{i}' for i in range(N_FEATURES_HASH)]
df_hashed_regions = pd.DataFrame(hashed_features_sparse.toarray(),
columns=hashed_feature_columns,
index=df_wines.index)


In [19]:
df_wines = pd.concat([df_wines.drop(['RegionName'], axis=1), df_hashed_regions], axis=1)

In [20]:
df_wines = df_wines.drop(['WineryName','WineName'], axis=1)

In [21]:
df_wines.head()

,WineID,Grapes,Harmonize,ABV,Code,RegionID,WineryID,Website,Vintages,Type_dessert,...,RegionHash_6,RegionHash_7,RegionHash_8,RegionHash_9,RegionHash_10,RegionHash_11,RegionHash_12,RegionHash_13,RegionHash_14,RegionHash_15
0,100001,[muscatmoscato],"[pork, rich fish, shellfish]",-1.774251,BR,1001,10001,http://www.vinicolaperini.com.br,"[2020, 2019, 2018, 2017, 2016, 2015, 2014, 201...",False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,100002,[ancellotta],"[beef, barbecue, codfish, pasta, pizza, cheese]",-0.408248,BR,1001,10001,http://www.vinicolaperini.com.br,"[2016, 2015, 2014, 2013, 2012, 2011, 2010, 200...",False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,100003,[cabernet sauvignon],"[beef, lamb, poultry]",-0.408248,BR,1001,10002,https://www.emporiocastellamare.com.br,"[2021, 2020, 2019, 2018, 2017, 2016, 2015, 201...",False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,100005,"[cabernet sauvignon, merlot]","[beef, lamb, game meat, poultry]",-0.711804,BR,1001,10000,http://www.vinicolaaurora.com.br,"[2021, 2020, 2019, 2018, 2017, 2016, 2015, 201...",False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,100007,[muscatmoscato bianco],"[pork, rich fish, shellfish]",-1.774251,BR,1001,10012,http://www.dalpizzol.com.br,"[2018, 2017, 2016, 2015, 2014, 2013, 2012, 201...",False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
from scipy.sparse import csr_matrix

non_text_columns = ['ABV'] + [col for col in df_wines.columns if col.startswith('type_')
or col.startswith('elaborate_')
or col.startswith('body_')
or col.startswith('acidity_')
or col.startswith('country_')]
non_text_columns = [col for col in non_text_columns if col not in hashed_feature_columns]

if non_text_columns:
    numeric_sparse = csr_matrix(df_wines[non_text_columns].values)
else:
    numeric_sparse = None


In [23]:
from scipy.sparse import hstack

matrices = []
if numeric_sparse is not None:
    matrices.append(numeric_sparse)
    matrices.append(tfidf_matrix)
    matrices.append(hashed_features_sparse)

combined_sparse = hstack(matrices)
print("Combined feature matrix shape:", combined_sparse.shape)

Combined feature matrix shape: (1007, 278)


In [24]:
combined_dense = combined_sparse.toarray()
embedding_df = pd.DataFrame(combined_dense, index=wine_ids)
embedding_df.index.name = "WineID"

In [25]:
embedding_df.head()

,0,1,2,3,4,5,6,7,8,9,...,268,269,270,271,272,273,274,275,276,277
WineID,,,,,,,,,,,,,,,,,,,,,
100001,-1.774251,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100002,-0.408248,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100003,-0.408248,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100005,-0.711804,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100007,-1.774251,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
from sklearn.metrics.pairwise import cosine_similarity

similarity_matrix = cosine_similarity(combined_sparse)

In [27]:
similarity_df = pd.DataFrame(similarity_matrix, index=wine_ids, columns=wine_ids)

In [28]:
similarity_df.head()

WineID,100001,100002,100003,100005,100007,100008,100010,100012,100013,100014,...,198580,198885,199204,199306,199360,199408,199481,199533,199885,200139
WineID,,,,,,,,,,,,,,,,,,,,,
100001,1.000000,0.516307,0.516307,0.629946,0.952912,0.285456,-0.108890,0.545759,-0.623514,-0.623514,...,0.081255,0.241784,0.601641,0.565256,0.019903,0.012275,-0.108890,0.317677,0.231239,0.057731
100002,0.516307,1.000000,0.555854,0.565764,0.516307,0.488982,-0.027780,0.189163,-0.208467,-0.208467,...,0.020476,0.094202,0.543265,0.176538,-0.009229,0.038551,-0.030144,0.066188,0.066188,0.070012
100003,0.516307,0.555854,1.000000,0.871542,0.516307,0.951008,0.072776,0.189163,-0.213585,-0.213585,...,0.020476,0.101021,0.573418,0.196291,0.013615,-0.009229,0.024082,0.104534,0.088385,0.091915
100005,0.629946,0.565764,0.871542,1.000000,0.629946,0.761955,0.014089,0.306634,-0.353273,-0.353273,...,0.033191,0.141283,0.613129,0.299768,0.021178,0.001845,0.031678,0.133691,0.142405,0.082375
100007,0.952912,0.516307,0.516307,0.629946,1.000000,0.285456,-0.108890,0.542749,-0.623514,-0.623514,...,0.075553,0.235748,0.587169,0.548891,0.008771,0.002992,-0.108890,0.285907,0.220422,0.057731


In [29]:
# --- Calculate Wine Statistics (for Cold Start) ---
print("Calculating wine statistics...")
if not df_ratings.empty:
    # Basic stats: count and mean rating
    wine_stats = df_ratings.groupby('WineID')['Rating'].agg(['count', 'mean']).reset_index()
    wine_stats.rename(columns={'count': 'rating_count', 'mean': 'rating_mean'}, inplace=True)

    # Merge with original wine names and key context for better cold start info
    context_cols_to_merge = ['WineID', 'WineName', 'Country', 'RegionName', 'WineryName', 'Type']
    valid_context_cols = [col for col in context_cols_to_merge if col in df_wines.columns]

    if 'WineID' in df_wines.columns:
         wine_stats = pd.merge(wine_stats, df_wines[valid_context_cols], on='WineID', how='left')
         # Fill missing context info (for wines potentially missing in original file after merge)
         for col in valid_context_cols:
             if col != 'WineID' and wine_stats[col].isnull().any():
                 wine_stats[col].fillna('Unknown', inplace=True)
    print(f"Wine statistics calculated for {len(wine_stats)} wines.")
else:
    print("Warning: Ratings data is empty. Cannot calculate wine statistics.")
    wine_stats = pd.DataFrame(columns=['WineID', 'rating_count', 'rating_mean']) # Empty df

Calculating wine statistics...
Wine statistics calculated for 1007 wines.


In [30]:
def get_recommendation(target_wine_id, similarity_df, num_recs=5):
    """CONTENT-BASED: Gets wines most similar to the target wine."""
    if similarity_df.empty or target_wine_id not in similarity_df.index:
        # print(f"Warning: WineID {target_wine_id} or similarity data not found.")
        return pd.Series(dtype='float64')
    try:
        sim_scores = similarity_df.loc[target_wine_id]
        recs = sim_scores.nlargest(num_recs + 1)
        recs = recs[recs.index != target_wine_id] # Exclude self
        return recs.head(num_recs)
    except KeyError:
        # print(f"Warning: KeyError looking up WineID {target_wine_id} in similarity data.")
        return pd.Series(dtype='float64')

In [31]:
def predict_rating(user_id, target_wine_id, df_ratings, similarity_df, k=10):
    """CONTENT-BASED: Predicts rating based on user's ratings of similar items."""
    # Use .get() for safer access, handles if target_wine_id is not a column/index
    target_similarities = similarity_df.get(target_wine_id)

    if target_similarities is None: # Check if wine exists in similarity matrix
        # print(f"Warning: Wine {target_wine_id} not found in similarity data for prediction.")
        user_ratings_check = df_ratings[df_ratings['UserID'] == user_id]['Rating']
        return user_ratings_check.mean() if not user_ratings_check.empty else None

    user_ratings = df_ratings[df_ratings['UserID'] == user_id].set_index('WineID')['Rating']
    if user_ratings.empty: return None # Cannot predict without ratings

    rated_indices = user_ratings.index.intersection(target_similarities.index)
    rated_indices = rated_indices.drop(target_wine_id, errors='ignore')

    if rated_indices.empty: return user_ratings.mean() # Fallback to user average

    similarities_of_rated = target_similarities.loc[rated_indices]
    positive_similarities = similarities_of_rated[similarities_of_rated > 0]
    if positive_similarities.empty: return user_ratings.mean() # Fallback

    actual_k = min(k, len(positive_similarities))
    top_k_neighbors = positive_similarities.nlargest(actual_k)

    neighbor_ratings = user_ratings.loc[top_k_neighbors.index]
    weighted_sum = (top_k_neighbors * neighbor_ratings).sum()
    sum_of_weights = top_k_neighbors.sum()

    if sum_of_weights == 0: return user_ratings.mean() # Fallback

    return weighted_sum / sum_of_weights

In [32]:
def get_embedding_preference_recs(preferences, df_prefs_data, similarity_df, num_representatives=15, num_recs=10):
    """COLD START (User): Uses average embedding profile based on preferences."""
    if df_prefs_data is None or df_prefs_data.empty: return pd.DataFrame()
    if similarity_df is None or similarity_df.empty: return pd.DataFrame()

    # --- Step 1: Find representative wines (copied logic from get_preference_based_wines) ---
    if 'WineID' in df_prefs_data.columns: matches_df = df_prefs_data.set_index('WineID')
    else: matches_df = df_prefs_data
    possible_indices = matches_df.index
    for key, value in preferences.items():
        if key not in matches_df.columns: continue
        if isinstance(value, str): pref_values = [value.strip().lower()]
        elif isinstance(value, list): pref_values = [str(v).strip().lower() for v in value if v]
        else: continue
        if not pref_values: continue
        col_data = matches_df.loc[possible_indices, key]
        try:
            first_valid = col_data.dropna().iloc[0] if not col_data.dropna().empty else None
            is_list_col = isinstance(first_valid, list)
        except IndexError: is_list_col = False
        if is_list_col: is_match = col_data.apply(lambda wl: isinstance(wl, list) and any(p in wl for p in pref_values))
        else: is_match = col_data.astype(str).str.lower().isin(pref_values)
        possible_indices = possible_indices.intersection(possible_indices[is_match])
        if possible_indices.empty: break
    representative_wine_ids = possible_indices.tolist()
    # --- End Step 1 ---

    if not representative_wine_ids: return pd.DataFrame()

    if len(representative_wine_ids) > num_representatives:
        representative_wine_ids = representative_wine_ids[:num_representatives]

    valid_representative_ids = [wid for wid in representative_wine_ids if wid in similarity_df.index]
    if not valid_representative_ids: return pd.DataFrame()

    # --- Step 3: Average similarity profiles ---
    representative_profiles = similarity_df.loc[valid_representative_ids]
    preference_similarity_vector = representative_profiles.mean(axis=0)

    # --- Step 4 & 5: Find most similar and filter representatives ---
    num_to_fetch = num_recs + len(valid_representative_ids)
    top_matches = preference_similarity_vector.nlargest(num_to_fetch)
    final_recommendations = top_matches[~top_matches.index.isin(valid_representative_ids)]

    # --- Step 6: Format ---
    recs_df = final_recommendations.head(num_recs).reset_index()
    recs_df.columns = ['WineID', 'similarity_to_preference']

    # Note: Merging names here depends on df_wines_original being available globally or passed in.
    # It's better practice to pass it in, as done in the 'recommend_for_user' function.

    return recs_df

In [33]:
def check_if_user_has_ratings(user_id, df_ratings):
    """Checks if a user exists in the ratings data."""
    if df_ratings is None or df_ratings.empty: return False
    return (df_ratings['UserID'] == user_id).any()

def find_candidate_items_for_user(user_id, all_wine_ids, df_ratings):
    """Finds wine IDs user hasn't rated (can be improved with more complex logic)."""
    if df_ratings is None or df_ratings.empty: return list(all_wine_ids)
    rated_by_user = set(df_ratings[df_ratings['UserID'] == user_id]['WineID'].unique())
    candidates = [wine_id for wine_id in all_wine_ids if wine_id not in rated_by_user]
    return candidates

def rank_items_using_predict_rating(user_id, candidate_items, df_ratings, similarity_df, k=10, num_recs=10):
    """Predicts ratings for candidates and returns top N ranked."""
    predictions = {}
    # print(f"  Predicting ratings for {len(candidate_items)} candidate items...") # Verbose
    for wine_id in candidate_items:
        predicted = predict_rating(user_id, wine_id, df_ratings, similarity_df, k=k)
        if predicted is not None: predictions[wine_id] = predicted
    if not predictions: return pd.DataFrame()
    sorted_candidates = sorted(predictions.items(), key=lambda item: item[1], reverse=True)
    recs_df = pd.DataFrame(sorted_candidates, columns=['WineID', 'predicted_rating']).head(num_recs)
    # print(f"  Ranking complete. Returning top {len(recs_df)} items.") # Verbose

In [34]:
def recommend_for_user(user_id,
                       df_ratings,
                       similarity_df,
                       df_for_prefs,
                       wine_stats,
                       df_wines_original, # Pass original df for reliable context/name merging
                       preferences=None,
                       num_recs=10,
                       k_neighbors_pred=15,
                       num_representatives_embed=15 # Use config value
                       ):
    """
    Master function: provides recommendations, handling cold-start.
    MODIFIED COLD START: Only uses embedding prefs if preferences provided,
                         then falls back to trending/popular.
    """
    print(f"\n--- Generating recommendations for UserID: {user_id} ---")
    all_wine_ids = similarity_df.index if similarity_df is not None and not similarity_df.empty else []
    if not list(all_wine_ids):
         print("ERROR: No wine IDs available in similarity matrix. Cannot recommend.")
         return pd.DataFrame()

    user_has_history = check_if_user_has_ratings(user_id, df_ratings)
    final_recs = pd.DataFrame()
    reason = "Unknown"

    if user_has_history:
        # --- Personalized Strategy (No changes here) ---
        print("User has rating history. Using personalized strategy (predict ratings).")
        candidate_items = find_candidate_items_for_user(user_id, all_wine_ids, df_ratings)
        recs_pred = rank_items_using_predict_rating(user_id, candidate_items, df_ratings, similarity_df, k=k_neighbors_pred, num_recs=num_recs)
        final_recs = recs_pred
        reason = 'Match Predicted Ratings'
        print("  Success using predicted ratings.")
    else:
        # --- Cold Start Strategy (MODIFIED LOGIC) ---
        print("User is new or has no history. Using cold start strategy.")
        cold_start_success = False

        # --- Step 1: Try Embedding Preferences ONLY if preferences are provided ---
        if preferences:
            print(f"  User provided preferences: {preferences}")
            print("  Trying embedding preference recommendations...")
            recs_embed = get_embedding_preference_recs(preferences,
                                                       df_for_prefs,
                                                       similarity_df,
                                                       num_representatives=num_representatives_embed,
                                                       num_recs=num_recs)
            if not recs_embed.empty:
                final_recs = recs_embed
                # Ensure 'WineName' is present (get_embedding_preference_recs should add it, but double-check)
               
                reason = 'Match Embedded Preference Profile'
                cold_start_success = True
                print("  Success using embedding preferences.")
            else:
                print("  Embedding preference method failed to generate recommendations.")
        else:
            print("  No preferences provided by the user.")

       

        if not cold_start_success:
            print("  ERROR: All cold start methods failed to generate recommendations.")
            # final_recs remains an empty DataFrame

    # --- Final Formatting ---
    if not final_recs.empty:
        final_recs['reason'] = reason
     

    return final_recs

In [35]:
data_ready = True
required_dfs = {
'df_ratings': df_ratings,
'similarity_df': similarity_df,
'df_for_prefs': df_for_prefs,
'wine_stats': wine_stats,
'df_wines_original': df_wines
}

for name, df_var in required_dfs.items():
    if not isinstance(df_var, pd.DataFrame) or df_var.empty:
        print(f"ERROR: Required DataFrame '{name}' is missing or empty. Cannot run examples.")
        data_ready = False
        break

if data_ready:
    new_user_id_with_prefs = 999998   # New user ID
    user_prefs_example = {
    'Type': 'white',
    'Country': 'chile'
    }
    print(f"\nTesting Scenario 3: New User with Preferences (UserID: {new_user_id_with_prefs})")
    print(f"Preferences: {user_prefs_example}")
    recs_new_with_prefs = recommend_for_user(new_user_id_with_prefs, df_ratings, similarity_df,
    df_for_prefs, wine_stats, df_wines,
    preferences=user_prefs_example, num_recs=5)
    if not recs_new_with_prefs.empty:
        print("\nRecommendations for the new user:")
        print(recs_new_with_prefs)
    else:
        print("Could not generate recommendations for new user with preferences.")
else:
    print("\nSkipping Master Recommender examples due to missing data.")


Testing Scenario 3: New User with Preferences (UserID: 999998)
Preferences: {'Type': 'white', 'Country': 'chile'}

--- Generating recommendations for UserID: 999998 ---
User is new or has no history. Using cold start strategy.
  User provided preferences: {'Type': 'white', 'Country': 'chile'}
  Trying embedding preference recommendations...
  Success using embedding preferences.

Recommendations for the new user:
   WineID  similarity_to_preference                             reason
0  164945                  0.455616  Match Embedded Preference Profile
1  187681                  0.453140  Match Embedded Preference Profile
2  179667                  0.451350  Match Embedded Preference Profile
3  171246                  0.436449  Match Embedded Preference Profile
4  199533                  0.418347  Match Embedded Preference Profile


In [36]:
# Example: Get item-to-item recommendations for the first wine
if not wine_ids.empty:
    example_wine_id = wine_ids.iloc[0]
    print(f"\n--- Item-to-Item Recommendations for WineID: {example_wine_id} ---")
    recommendations = get_recommendation(example_wine_id, similarity_df, num_recs=5)
    print(recommendations)
else:
    print("\nWine IDs list is empty, skipping item-to-item recommendation example.")


--- Item-to-Item Recommendations for WineID: 100001 ---
WineID
100073    1.000000
100061    0.999034
100007    0.952912
100088    0.902500
184497    0.832754
Name: 100001, dtype: float64


In [37]:
# Example: Predict rating for a user and a wine
# Find a user and an item they haven't rated yet (for a meaningful prediction)
if not df_ratings.empty and not wine_ids.empty:
    example_user_id = df_ratings['UserID'].iloc[0] # Take the first user
    rated_by_user = df_ratings[df_ratings['UserID'] == example_user_id]['WineID'].tolist()
    # Find a wine ID present in similarity_df but not rated by this user
    potential_items_to_predict = similarity_df.index.difference(rated_by_user)

    if not potential_items_to_predict.empty:
        example_target_wine_id = potential_items_to_predict[0] # Take the first unrated item

        print(f"\n--- Rating Prediction for UserID: {example_user_id}, WineID: {example_target_wine_id} ---")
        predicted = predict_rating(example_user_id, example_target_wine_id, df_ratings, similarity_df, k=10)

        if predicted is not None:
            print(f"Predicted Rating: {predicted:.4f}")
        else:
            print("Could not predict rating.")
    else:
        print(f"\nUser {example_user_id} seems to have rated all available wines, cannot demonstrate prediction.")
else:
     print("\nRatings data or wine IDs are empty, skipping rating prediction example.")


--- Rating Prediction for UserID: 1356810, WineID: 100001 ---
Predicted Rating: 4.0142
